In [ ]:


"""
学習データ：
    カテゴリ：馬名、騎手名、馬齢
        レース名、競馬場名、天気、馬場の種類、馬場状態、開催ラウンド
    数値：そのレースの賞金、距離
    
教師データ：着順

netkeiba.comよりスクレイピング
https://cocon-corporation.com/cocontoco/horseraceprediction_ai_lambdarank/#index4
こちら参考にLambdaRankアルゴリズム・lightGBMで学習させる。


ssh 2015

---
sudo yum install python3

---
pip3 install --user pandas selenium bs4 lxml html5lib
sudo yum -y install https://dl.google.com/linux/direct/google-chrome-stable_current_x86_64.rpm
wget https://chromedriver.storage.googleapis.com/80.0.3987.106/chromedriver_linux64.zip
unzip chromedriver_linux64.zip -d bin/
vi 2020.py

---
貼り付け
---
nohup python3 2020.py &
exit

---
→最初へ

取得後：
---
scp  2020:/home/ec2-user/horses_data2020.csv ~/Horse_racing_prediction
scp  2019:/home/ec2-user/horses_data2019.csv ~/Horse_racing_prediction
scp  2018:/home/ec2-user/horses_data2018.csv ~/Horse_racing_prediction
scp  2017:/home/ec2-user/horses_data2017.csv ~/Horse_racing_prediction
scp  2016:/home/ec2-user/horses_data2016.csv ~/Horse_racing_prediction
scp  2015:/home/ec2-user/horses_data2015.csv ~/Horse_racing_prediction

--------
コード直したら
ssh 

rm 2020.py
vi 2020.py

貼り付け

nohup python3 2020.py &
exit

"""



In [ ]:
import traceback
import pandas as pd
import urllib.parse
import numpy as np
import urllib.request
from bs4 import BeautifulSoup
import re
from time import sleep 
import warnings
warnings.simplefilter('ignore')
def make_data(race_id):
    target_url = 'https://race.netkeiba.com/race/result.html?race_id='+race_id+'&rf=race_list'
    dfs = pd.read_html(target_url)
    df = dfs[0]
    df_1 = dfs[1]
    df_2 = dfs[2]
    odds_all = []
    for df_ in [df_1,df_2]:
        for i in range(len(df_)):
            l = []
            for j in range(3):
                l.append(df_.iloc[i,j])

            odds_all.append("|".join(l))

    odds_all = ";".join(odds_all)
    with urllib.request.urlopen(target_url) as response:
        html = response.read()
        # 馬場距離天気
        soup = BeautifulSoup(html, "html.parser")
   
    #15:45発走 / 芝1600m (左 外)\n/ 天候:晴\n/ 馬場:良
    data_1 = soup.select_one("#page > div.RaceColumn01 > div > div.RaceMainColumn > div.RaceList_NameBox > div.RaceList_Item02 > div.RaceData01").text
    result_1 = re.findall(r'/\s([^\s]+)',data_1)
    siba_da = result_1[0][0]
    length = float(result_1[0][1:-1])
    weather = result_1[1][-1]
    baba = result_1[2][-1]
    # 本賞金:3900,1600,980,590,390万円
    data_2 = soup.select_one("#page > div.RaceColumn01 > div > div.RaceMainColumn > div.RaceList_NameBox > div.RaceList_Item02 > div.RaceData02 > span:nth-child(10)").text
    result_2 = re.findall(r'本賞金:([^,]+),',data_2)
    prize = float(result_2[0])
    # 着順・馬名・性齢・騎手名
    df["着順"] = [i for i in range(1,len(df)+1)]
    df["馬齢"] = [x[1] for x in df["性齢"]]
    df = df[["着順","馬番","馬名","馬齢","騎手","単勝オッズ"]]
    df = df.sort_values('馬番')
    new_columns = ["レース番号","芝/ダ","距離","天気","馬場","賞金","odds_all"]
    new_values = [race_id,siba_da,length,weather,baba,prize,odds_all]
    for c,v in zip(new_columns,new_values):
        df[c] = v
    
    return df

def make_csv(year):
    where = [str(i )for i in range(1,10)]
    kaisai = [str(i )for i in range(1,6)]
    day = [str(i )for i in range(1,10)]
    rounds = ["01","02","03","04","05","06","07","08","09","10","11","12"]
    first = 1
    for y in [year]:
        for w in where:
            print(w)
            for k in kaisai:
                for d in day:
                    for r in rounds:
                        race_id = y+"0"+w+"0"+k+"0"+d+r
                        try:
                            df = make_data(race_id)
                            
                        except ValueError as e:
                         
                            continue
                        except Exception as e:
                            print(race_id+";失敗of:Exception" + str(e))
                            traceback.print_exc()
                            continue
                            
                       
                        if first == 1:
                            df.to_csv("horses_data_" + y +".csv")
                            first = 0
                        else:
                            df.to_csv("horses_data_" + y +".csv", mode='a',header=False)

year_list = ["2015","2016"]
for i in year_list:
    make_csv(i)

In [1]:
%%time
import traceback
import pandas as pd
import numpy as np
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
import re
from time import sleep 
import warnings
warnings.simplefilter('ignore')
from concurrent import futures

def id2kaisai(race_id):
    num = int(race_id[4:6])
    if num == 1:
        return race_id[0:4]+race_id[7]+"札幌"+race_id[9]
    elif num == 2:
        return race_id[0:4]+race_id[7]+"函館"+race_id[9]
    elif num == 3:
        return race_id[0:4]+race_id[7]+"福島"+race_id[9]
    elif num == 4:
        return race_id[0:4]+race_id[7]+"新潟"+race_id[9]
    elif num == 5:
        return race_id[0:4]+race_id[7]+"東京"+race_id[9]
    elif num == 6:
        return race_id[0:4]+race_id[7]+"中山"+race_id[9]
    elif num == 7:
        return race_id[0:4]+race_id[7]+"中京"+race_id[9]
    elif num == 8:
        return race_id[0:4]+race_id[7]+"京都"+race_id[9]
    elif num == 9:
        return race_id[0:4]+race_id[7]+"阪神"+race_id[9]
    else:
        return race_id[0:4]+race_id[7]+"小倉"+race_id[9]
    
def get_table(target_url):
    with urllib.request.urlopen(target_url) as response:
        html = response.read()
        return pd.read_html(html)
    
def make_data(race_id):
    target_url = 'https://race.netkeiba.com/race/result.html?race_id='+race_id+'&rf=race_list'
    dfs = pd.read_html(target_url)
    df = dfs[0]
    df_1 = dfs[1]
    df_2 = dfs[2]
    odds_all = []
    for df_ in [df_1,df_2]:
        for i in range(len(df_)):
            l = []
            for j in range(3):
                l.append(df_.iloc[i,j])

            odds_all.append("|".join(l))

    odds_info = ";".join(odds_all).replace(",","")
    with urllib.request.urlopen(target_url) as response:
        html = response.read()
        # 馬場距離天気
        soup = BeautifulSoup(html, "html.parser")
   
    #レース情報作成
    # "race_id;siba_da;length;weather;baba;prize"
    data_1 = soup.select_one("#page > div.RaceColumn01 > div > div.RaceMainColumn > div.RaceList_NameBox > div.RaceList_Item02 > div.RaceData01").text
    result_1 = re.findall(r'/\s([^\s]+)',data_1)
    siba_da = result_1[0][0]
    length = result_1[0][1:-1]
    weather = result_1[1][-1]
    baba = result_1[2][-1]
    
    data_2 = soup.select_one("#page > div.RaceColumn01 > div > div.RaceMainColumn > div.RaceList_NameBox > div.RaceList_Item02 > div.RaceData02 > span:nth-child(10)").text
    result_2 = re.findall(r'本賞金:([^,]+),',data_2)
    prize = result_2[0]
    # 着順・馬名・性齢・騎手名
    
    #new_columns = ["レース番号","芝/ダ","距離","天気","馬場","賞金","odds_all"]
    list_race = [race_id,siba_da,length,weather,baba,prize]
    race_info = ";".join(list_race)
    
    #各馬情報作成
    # "着順;馬番;馬名;馬齢;騎手;単勝オッズ;調教師;馬主;生産者;父馬;母馬;それまでの獲得賞金
    df["着順"] = [i for i in range(1,len(df)+1)]
    df["馬齢"] = [x[1] for x in df["性齢"]]
    df["馬体重"] = [x[0:3] for x in df["馬体重(増減)"]]
    df["増減"] = [x[4:-1] for x in df["馬体重(増減)"]]
    df = df[["着順","馬番","馬名","馬齢","騎手","単勝オッズ","馬体重","増減"]]
    df = df.sort_values('馬番')
    horse_urls = []
    for i in range(len(df)):
        horse_url = soup.select_one("#All_Result_Table > tbody > tr:nth-child("+str(i+1)+") > td:nth-child(4) > span > a").get("href") 
        
        horse_urls.append(horse_url)
        
    with futures.ThreadPoolExecutor(max_workers=4) as executor:
        results = list(executor.map(get_table,horse_urls))
        
    horses_list = []
    for i in range(len(df)):
        horse_list = list(map(str,df.iloc[i,:].values))
        table_0 = results[i][1]
    
        table_0.index = table_0.iloc[:,0]
        horse_list.append(table_0.loc["調教師",1])
        horse_list.append(table_0.loc["馬主",1])
        horse_list.append(table_0.loc["生産者",1])
        table_1 = results[i][2]
        horse_list.append(table_1.loc[0,0])
    
        horse_list.append(table_1.loc[2,0])
        
        table_2 = results[i][3]
        
        if table_2.columns[0] == "受賞歴":
            table_2 = results[i][4]
            
        table_2.index = [table_2.iloc[i,0][0:4]+table_2.iloc[i,1] for i in range(len(table_2))]
        a = table_2.index.get_loc(id2kaisai(race_id))
        table_2 = table_2.iloc[a+1:,:]
        #table_2["賞金"].fillna('0',inplace=True)
        #table_2["賞金"] = list(map(lambda x: x.replace(",",""),table_2["賞金"]))
        horse_list.append(str(np.sum(table_2["賞金"])))
        horse_info = ";".join(horse_list)
        horses_list.append(horse_info)
    horses_info = ",".join(horses_list)
    
    info = ",".join([race_info,horses_info,odds_info])+"\n"
    return info

def get_text(race_id):
    try:
        text = make_data(race_id)
        return text
    except Exception as e:
        return 0
    

                            

    
race_id_list = []
where = [str(i )for i in range(1,10)]
kaisai = [str(i )for i in range(1,6)]
day = [str(i )for i in range(1,10)]
rounds = ["01","02","03","04","05","06","07","08","09","10","11","12"]
year_list = ["2020"]
for y in year_list:
    for w in where:
        print(w)
        for k in kaisai:
            for d in day:
                for r in rounds:
                    race_id = y+"0"+w+"0"+k+"0"+d+r
                    race_id_list.append(race_id)

with futures.ThreadPoolExecutor(max_workers=4) as executor:
        results = list(executor.map(get_text,race_id_list))
with open("keiba_2020", mode='w') as f:
    pass
with open("keiba_2020", mode='a') as f:
    for text in result:
        f.write(text)

1
2
3
4
5
6
7
8
9


NameError: name 'result' is not defined

In [2]:
with open("keiba_2020", mode='w') as f:
    pass
with open("keiba_2020", mode='a') as f:
    for text in results:
        if text != 0:
            f.write(text)

In [35]:
df = pd.read_html("https://db.netkeiba.com/horse/2010103783")[3]
df

,受賞歴
0,JRA賞最優秀2歳牡馬(2012年)


In [6]:
driver = Chrome()
driver.get("https://www.google.com/search?rlz=1C5CHFA_enJP835JP835&sxsrf=ALeKk00hf0eDPelBxCiMDwg7tUtKWocX5g%3A1583320751541&ei=r45fXvjcINn7wAOqkoKgCw&q=selenium+python+%E3%82%AF%E3%83%AD%E3%83%BC%E3%83%AA%E3%83%B3%E3%82%B0&oq=selenium+python+%E3%82%AF%E3%83%AD%E3%83%BC%E3%83%AA%E3%83%B3%E3%82%B0&gs_l=psy-ab.3..0i8i30.4044.9171..9816...0.0..0.179.1697.2j12......0....1..gws-wiz.......35i39j0j0i4j0i4i30j0i8i4i30j0i30.rUPcK7Ffp8I&ved=0ahUKEwi48Ne32YDoAhXZPXAKHSqJALQQ4dUDCAs&uact=5")
driver.get("https://www.google.com/search?rlz=1C5CHFA_enJP835JP835&sxsrf=ALeKk00hf0eDPelBxCiMDwg7tUtKWocX5g%3A1583320751541&ei=r45fXvjcINn7wAOqkoKgCw&q=selenium+python+%E3%82%AF%E3%83%AD%E3%83%BC%E3%83%AA%E3%83%B3%E3%82%B0&oq=selenium+python+%E3%82%AF%E3%83%AD%E3%83%BC%E3%83%AA%E3%83%B3%E3%82%B0&gs_l=psy-ab.3..0i8i30.4044.9171..9816...0.0..0.179.1697.2j12......0....1..gws-wiz.......35i39j0j0i4j0i4i30j0i8i4i30j0i30.rUPcK7Ffp8I&ved=0ahUKEwi48Ne32YDoAhXZPXAKHSqJALQQ4dUDCAs&uact=5")


In [17]:
driver.close()

In [16]:
race_id = "201705030111"
target_url = 'https://race.netkeiba.com/race/result.html?race_id='+race_id+'&rf=race_list'
df_1 = pd.read_html(target_url)[1]
df_2 = pd.read_html(target_url)[2]
odds = []
for df in [df_1,df_2]:
    for i in range(len(df)):
        l = []
        for j in range(3):
            l.append(df.iloc[i,j])
        
        odds.append("|".join(l))

odds = ";".join(odds)
print(odds)
            
           
            


単勝|6|190円;複勝|6 5 9|130円430円350円;枠連|3 3|2,000円;馬連|5 6|1,870円;ワイド|5 6 6 9 5 9|720円610円3,010円;馬単|6 5|2,560円;3連複|5 6 9|7,200円;3連単|6 5 9|25,490円
